# Splink

In [1]:
from splink.spark.jar_location import similarity_jar_location
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql.functions import regexp_replace, trim, col, upper

import pyspark.sql.functions as sf

# conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
# conf.set("spark.default.parallelism", "16")
# conf.set("spark.sql.shuffle.partitions", "50")

# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
# conf.set("spark.jars", path)

configuration_cluster = (
    SparkConf()
    .set("spark.executor.cores", "4")
    .set("spark.dynamicAllocation.maxExecutors", "20")
    .set("spark.executor.memory", "20g")
    .set("spark.driver.memory", "16g")
    .set("spark.driver.maxResultSize", "8g")
    .set("spark.sql.shuffle.partitions", "200")
    .set("spark.kryoserializer.buffer.max", "1g")
    .set("spark.dynamicAllocation.enabled", "true")
    .set("spark.network.timeout", "180000")
    .set("spark.sql.execution.arrow.pyspark.enabled", "true")
    .set("parquet.enable.dictionary","false")
    .set("spark.jars", path)
)

#Create SparkSession
spark = (
    SparkSession.builder.appName("matchingSplink")
    .config(conf=configuration_cluster)
    .getOrCreate()
)

# spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

24/02/11 18:28:28 WARN Utils: Your hostname, genex resolves to a loopback address: 127.0.1.1; using 192.168.4.13 instead (on interface eth2)
24/02/11 18:28:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/02/11 18:28:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/11 18:28:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
import time
start = time.time()

In [2]:
dataset1 = spark.read.parquet("/home/colombo/BusinessRegistryTool/Data/mfi/parquet/")\
                         .select(col("code").alias("unique_id"), "name", "country", "address", "post", "city")#.filter(col('country') == 'CZ')
# dataset1.cache()
# dataset1.count()
dataset2 = spark.read.parquet("/home/colombo/BusinessRegistryTool/Data/gleif/parquet")\
                         .select(col("LEI").alias("unique_id"), col("`Entity.LegalName`").alias("name"), col("`Entity.LegalAddress.Country`").alias("country"),\
                                 col("`Entity.LegalAddress.FirstAddressLine`").alias("address"), col("`Entity.LegalAddress.PostalCode`").alias("post"), \
                                col("`Entity.LegalAddress.City`").alias("city"))#.filter(col('country') == 'CZ')
# dataset2.cache()
# dataset2.count()

In [74]:
import splink.spark.comparison_library as cl
import splink.spark.comparison_template_library as ctl
from splink.spark.blocking_rule_library import block_on

settings = {
    "link_type": "link_only",
    "comparisons": [
        # cl.exact_match("name"),
        cl.levenshtein_at_thresholds("name", 3),
        cl.levenshtein_at_thresholds("address",4),
        # cl.exact_match("city", term_frequency_adjustments=True),
        # ctl.postcode_comparison("post"),
        cl.exact_match("country"),
    ],
    "blocking_rules_to_generate_predictions": [
        # block_on("city")
        # block_on("country")
        # block_on(["name"]),
        """(levenshtein(r.name,l.name) < 5) and r.country = l.country"""
    ],
    
    # "blocking_rules_to_generate_predictions": [
    #     block_on(["country"])
    # ],
    "em_convergence": 0.01
}

In [75]:
from splink.spark.linker import SparkLinker
linker = SparkLinker(
    [dataset1, dataset2],
    settings,
    spark=spark,
    input_table_aliases=["l", "r"],
    break_lineage_method="parquet")

24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function jaro_sim replaced a previously registered function.
24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function jaro_winkler replaced a previously registered function.
24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function jaccard replaced a previously registered function.
24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function cosine_distance replaced a previously registered function.
24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function dmetaphone replaced a previously registered function.
24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function dmetaphonealt replaced a previously registered function.
24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function qgramtokeniser replaced a previously registered function.
24/02/11 18:40:26 WARN SimpleFunctionRegistry: The function damerau_levenshtein replaced a previously registered function.


In [7]:
# df_predict = linker.deterministic_link()

In [76]:
linker.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----
24/02/11 18:40:29 WARN DataSource: All paths were ignored:
  file:/home/colombo/BusinessRegistryTool/Competitors/tmp_checkpoints/8f37de12-5448-467a-a391-94ebc7cea79e/__splink__df_concat_with_tf_814e6636e

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - name (no m values are trained).
    - address (no m values are trained).
    - country (no m values are trained).


In [77]:
# linker.estimate_parameters_using_expectation_maximisation(block_on("country"))
linker.estimate_parameters_using_expectation_maximisation(block_on("(levenshtein(r.name,l.name) < 5) and r.country = l.country"))
# linker.estimate_parameters_using_expectation_maximisation(block_on("name"))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(LEVENSHTEIN(l.`name`, l.`name`) < 5) AND l.`country` = l.`country` = (LEVENSHTEIN(r.`name`, r.`name`) < 5) AND r.`country` = r.`country`

Parameter estimates will be made for the following comparison(s):
    - address

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - name
    - country
24/02/11 18:43:05 WARN DataSource: All paths were ignored:                      
  file:/home/colombo/BusinessRegistryTool/Competitors/tmp_checkpoints/8f37de12-5448-467a-a391-94ebc7cea79e/__splink__df_comparison_vectors_3cba64258
                                                                                
Iteration 1: Largest change in params was 0.964 in the m_probability of address, level `All other comparisons`
Iteration 2: Largest change in params was 0.00437 in the m_probability of address, level `All other comparisons`

EM co

<EMTrainingSession, blocking on (LEVENSHTEIN(l.`name`, l.`name`) < 5) AND l.`country` = l.`country` = (LEVENSHTEIN(r.`name`, r.`name`) < 5) AND r.`country` = r.`country`, deactivating comparisons name, country>

In [78]:
results = linker.predict(threshold_match_probability=0.8)

24/02/11 18:44:11 WARN DataSource: All paths were ignored:                      
  file:/home/colombo/BusinessRegistryTool/Competitors/tmp_checkpoints/8f37de12-5448-467a-a391-94ebc7cea79e/__splink__df_predict_fcd80fcc6

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'name':
    m values not fully trained
Comparison: 'country':
    m values not fully trained
The 'probability_two_random_records_match' setting has been set to the default value (0.0001). 
If this is not the desired behaviour, either: 
 - assign a value for `probability_two_random_records_match` in your settings dictionary, or 
 - estimate with the `linker.estimate_probability_two_random_records_match` function.


In [22]:
print(time.time()-start)

292.5962550640106


In [81]:
df = results.as_pandas_dataframe()

In [19]:
df = results.as_record_dict()

In [20]:
import pandas as pd
df = pd.DataFrame(df)

In [21]:
df-

""


In [ ]:
# results.as_pandas_dataframe().to_csv('/home/colombo/BusinessRegistryTool/Data/results/splink_block_name.csv', index = False)

In [12]:
end = time.time()

In [82]:
spark.catalog.clearCache()
spark.stop()